In [1]:
import clip
import torch
import torchvision
from pathlib import Path
from PIL import Image
import torch.nn.functional as f

In [2]:
TARGET_IMAGE = '../datasets/NDI_images/Integreted/Calculated/'
ORIGINAL_IMAGE = '../datasets/NDI_images/Integreted/Observed/'

In [2]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [3]:
model = clip.load('ViT-L/14', device='cuda')
inputs = torch.randn((1, 1, 200, 200), device='cuda')

RuntimeError: No CUDA GPUs are available

In [10]:
model[0].encode_image(inputs)

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x2b71f1cbf550>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [ ]:
model[0].named_parameters

In [14]:
inputs = torch.randn((1, 3, 224, 224), device='cuda')
model[0].encode_image(inputs).shape

torch.Size([1, 768])

In [15]:
model[1]

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x2b71f1cbf550>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [20]:
device = torch.device('cuda')

transforms = torchvision.transforms.Compose([torchvision.transforms.Resize(224), torchvision.transforms.ToTensor()])
target_tensor = []
for i in range(1, 185):
    target_tensor.append(
        transforms(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), f'{i}.jpg'))).convert('RGB')).unsqueeze(0))
target_tensor = torch.cat(target_tensor, dim=0)
target_tensor = target_tensor.cuda(device)

input_tensor = transforms(Image.open(str(Path.joinpath(Path(ORIGINAL_IMAGE), '1.jpg'))).convert('RGB')).unsqueeze(0)
input_tensor = input_tensor.cuda()

target_embeddings = model[0].encode_image(target_tensor)
input_embeddings = model[0].encode_image(input_tensor)

In [25]:
target_embeddings = f.normalize(target_embeddings, dim=1)
input_embeddings = f.normalize(input_embeddings, dim=1)

logits = torch.einsum('nc,ck->nk', [target_embeddings, input_embeddings.T])

In [27]:
torch.argmax(logits)

tensor(36, device='cuda:0')